In [1]:
# Importing necessary modules
import requests
from bs4 import BeautifulSoup
import re
from collections import Counter

In [2]:
# Setting up notebook to display multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Defining a function to extract the HTML class and tag key words to search for ingredients
def ingredient_class_tag(recipe_html):
    # HTML class
    ingredients_class = re.findall('class=".*[Ii]ngredients[^ ]*"', str(recipe_html))
    ingredients_class_dict = dict(Counter(ingredients_class))
    most_common_ingredients_class = ""
    max_value = 0
    for key, value in ingredients_class_dict.items():
        if value > max_value:
            most_common_ingredients_class = key
            max_value = value
        elif value == max_value:
            if "label" in key.lower():
                most_common_ingredients_class = key
    most_common_ingredients_class = re.sub('class=|"', '', most_common_ingredients_class)
    # HTML tag
    tag_list = re.findall(rf'<.*{most_common_ingredients_class}.*>', str(recipe_html))
    clean_tag_list = []
    for t in tag_list:
        tag = re.search('<.* class', t)
        tag = re.sub('<| class', '', tag[0])
        clean_tag_list.append(tag)
    ingredients_tag_dict = dict(Counter(clean_tag_list))
    most_common_ingredients_tag = ""
    max_value = 0
    for key, value in ingredients_tag_dict.items():
        if value > max_value:
            most_common_ingredients_tag = key
            max_value = value
    return (most_common_ingredients_class, most_common_ingredients_tag)

In [4]:
# Defining a function to extract the ingredients and directions from an online recipe
def recipe_extract(URL):
    # Getting HTML of selected recipe
    webpage = requests.get(URL)
    recipe_html = BeautifulSoup(webpage.content, "html.parser")
    # Extracting the HTML class and tag key words to search for ingredients
    most_common_ingredients_class, most_common_ingredients_tag = ingredient_class_tag(recipe_html)
    # Extracting the ingredients and storing in list                
    ingredients_text = recipe_html.find_all(most_common_ingredients_tag, class_ = most_common_ingredients_class)
    ingredients_list = []
    for ingredient in ingredients_text:
        ingredient = str(ingredient)
        ingredient = re.sub('</', "", ingredient)
        ingredient = re.sub(rf'<{most_common_ingredients_tag}|{most_common_ingredients_tag}>', "", ingredient)
        ingredient = re.sub('<', "", ingredient)
        ingredient = re.sub(rf'{most_common_ingredients_class}', "", ingredient)
        ingredient = re.sub(' class=', "", ingredient)
        ingredient = re.sub('"', "", ingredient)
        ingredient = re.sub('Deselect All', "", ingredient)
        ingredient = re.sub('data.*?>', "", ingredient)
        ingredient = re.sub('span>|span', "", ingredient)
        ingredient = re.sub('p>', "", ingredient)
        ingredient = re.sub('>', "", ingredient)
        ingredient = re.sub(' +', " ", ingredient)
        ingredient = re.sub('\n', "", ingredient)
        ingredient = ingredient.strip()
        if ingredient:
            ingredients_list.append(ingredient)
    if "foodnetwork" in URL:
        # Extracting directions and storing in list
        directions_text = recipe_html.find("div", class_ = "o-Method__m-Body")
        directions_list = []
        for direction in directions_text:
            direction = str(direction)
            direction = re.sub('<ol>|<li class="o-Method__m-Step">|</li>|</ol>|\n|  ', "", direction)
            direction = direction.strip().split(".")
            if len(direction) > 1:
                for d in direction:
                    directions_list.append(d.strip())
        while("" in directions_list):
            directions_list.remove("")
    elif "allrecipes" in URL:
        # Extracting directions and storing in list
        directions_text = recipe_html.find_all("p", class_ = "comp mntl-sc-block mntl-sc-block-html")
        directions_list = []
        for direction in directions_text:
            direction = str(direction)
            direction = re.sub('<p class="comp mntl-sc-block mntl-sc-block-html".*>|</p>', "", direction)
            direction = direction.strip().split(".")
            if len(direction) > 1:
                for d in direction:
                    directions_list.append(d.strip())
        while("" in directions_list):
            directions_list.remove("")        
    return (ingredients_list, directions_list)

In [5]:
ingredients, directions = recipe_extract("https://www.foodnetwork.com/recipes/food-network-kitchen/sweet-and-sour-couscous-stuffed-peppers-recipe-2121036")
print(ingredients)
print(directions)

['2 medium carrots, cut into chunks', '2 stalks celery, cut into chunks', '1 large shallot, cut into chunks', '1 1/2 tablespoons olive oil', '1/2 pound lean ground beef', '2 tablespoons plus 4 teaspoons tomato paste', '1/3 cup chopped fresh parsley, dill or a combination', '1/3 cup golden raisins', '2 tablespoons red wine vinegar', 'Kosher salt', '4 red, yellow, orange or green bell peppers or a mix of colors, halved lengthwise and seeded', '1/2 cup whole wheat couscous', '3/4 cup grated asiago cheese']
['Preheat the oven to 450 degrees F', 'Pulse the carrots, celery and shallots in a food processor until coarsely chopped', 'Heat 1 tablespoon of the olive oil in a large nonstick skillet over medium high', 'Add the chopped vegetables and cook, stirring frequently, until light golden and soft, 8 to 10 minutes', '(Add a splash of water if the mixture begins to stick', ') Add the ground beef and 4 teaspoons of the tomato paste and cook, breaking the mixture up with a wooden spoon, until br

In [6]:
ingredients, directions = recipe_extract("https://www.allrecipes.com/recipe/222002/chef-johns-stuffed-peppers/")
print(ingredients)
print(directions)

['1 cup uncooked long grain white rice', '2 cups water', '1 onion, diced', '1 tablespoon olive oil', '2 cups marinara sauce', '1 cup beef broth', '1 tablespoon balsamic vinegar', '¼ teaspoon crushed red pepper flakes', '1 pound lean ground beef', '¼ pound hot Italian pork sausage, casing removed', '1 (10 ounce) can diced tomatoes', '¼ cup chopped fresh Italian parsley', '4 cloves garlic, minced', '2 teaspoons salt', '1 teaspoon freshly ground black pepper', '1 pinch ground cayenne pepper', '1 cup finely grated Parmigiano-Reggiano cheese, plus more for topping', '4 large green bell peppers, halved lengthwise and seeded']
['Bring rice and water to a boil in a saucepan over high heat', 'Reduce heat to medium-low, cover, and simmer until the rice is tender, and the liquid has been absorbed, 20 to 25 minutes', 'Set the cooked rice aside', 'Meanwhile, preheat the oven to 375 degrees F (190 degrees C)', 'Make sauce: Cook onion and olive oil over medium heat until onion begins to soften, about